# Enabling App Insights for Services in Production
With this notebook, you can learn how to enable App Insights for standard service monitoring, plus, we provide examples for doing custom logging within a scoring files in a model.


## What does Application Insights monitor?
It monitors request rates, response times, failure rates, etc. For more information visit [App Insights docs.](https://docs.microsoft.com/en-us/azure/application-insights/app-insights-overview)


## What is different compared to standard production deployment process?
If you want to enable generic App Insights for a service run:
```python
aks_service= Webservice(ws, "aks-w-dc2")
aks_service.update(enable_app_insights=True)```
Where "aks-w-dc2" is your service name. You can also do this from the Azure Portal under your Workspace--> deployments--> Select deployment--> Edit--> Advanced Settings--> Select "Enable AppInsights diagnostics"

If you want to log custom traces, you will follow the standard deplyment process for AKS and you will:
1. Update scoring file.
2. Update aks configuration.
3. Deploy the model with this new configuration. 

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/enable-app-insights-in-production-service/enable-app-insights-in-production-service.png)

## 1. Import your dependencies

In [ ]:
import azureml.core
import json

from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AksWebservice

print(azureml.core.VERSION)

## 2. Set up your configuration and create a workspace


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## 3. Register Model
Register an existing trained model, add descirption and tags.

In [ ]:
from azureml.core import Model

model = Model.register(model_path="sklearn_regression_model.pkl", # This points to a local file.
                       model_name="sklearn_regression_model.pkl", # This is the name the model is registered as.
                       tags={'area': "diabetes", 'type': "regression"},
                       description="Ridge regression model to predict diabetes",
                       workspace=ws)

print(model.name, model.description, model.version)

## 4. *Update your scoring file with custom print statements*
Here is an example:
### a. In your init function add:
```python
print ("model initialized" + time.strftime("%H:%M:%S"))```

### b. In your run function add:
```python
print ("Prediction created" + time.strftime("%H:%M:%S"))```

In [ ]:
%%writefile score.py
import os
import pickle
import json
import numpy
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
import time

def init():
    global model
    #Print statement for appinsights custom traces:
    print ("model initialized" + time.strftime("%H:%M:%S"))

    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')

    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)


# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        print ("Prediction created" + time.strftime("%H:%M:%S"))
        # you can return any datatype as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        print (error + time.strftime("%H:%M:%S"))
        return error

## 5. *Create myenv.yml file*
Please note that you must indicate azureml-defaults with verion >= 1.0.45 as a pip dependency, because it contains the functionality needed to host the model as a web service.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy==1.19.5','scikit-learn==0.22.1'],
                                 pip_packages=['azureml-defaults'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

## 6. Create Inference Configuration

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

## Deploy to ACI (Optional)

In [ ]:
from azureml.core.webservice import AciWebservice

aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                           memory_gb=1,
                                                           tags={'area': "diabetes", 'type': "regression"},
                                                           description="Predict diabetes using regression model",
                                                           enable_app_insights=True)

In [ ]:
aci_service_name = "aci-service-appinsights"

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output=True)

print(aci_service.state)

In [ ]:
if aci_service.state == "Healthy":
    test_sample = json.dumps({
        "data": [
            [1,28,13,45,54,6,57,8,8,10],
            [101,9,8,37,6,45,4,3,2,41]
        ]
    })

    prediction = aci_service.run(test_sample)

    print(prediction)
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aci_service.error)

## 7. Deploy to AKS service

### Create AKS compute if you haven't done so.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException

aks_name = "my-aks-insights"

creating_compute = False
try:
    aks_target = ComputeTarget(ws, aks_name)
    print("Using existing AKS compute target {}.".format(aks_name))
except ComputeTargetException:
    print("Creating a new AKS compute target {}.".format(aks_name))

    # Use the default configuration (can also provide parameters to customize).
    prov_config = AksCompute.provisioning_configuration()
    aks_target = ComputeTarget.create(workspace=ws,
                                      name=aks_name,
                                      provisioning_configuration=prov_config)
    creating_compute = True

In [ ]:
%%time
if creating_compute and aks_target.provisioning_state != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

In [ ]:
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

If you already have a cluster you can attach the service to it:

```python
%%time
resource_id = '/subscriptions/<subscriptionid>/resourcegroups/<resourcegroupname>/providers/Microsoft.ContainerService/managedClusters/<aksservername>'
create_name= 'myaks4'
attach_config = AksCompute.attach_configuration(resource_id=resource_id)
aks_target = ComputeTarget.attach(workspace=ws,
                                  name=create_name,
                                  attach_configuration=attach_config)
## Wait for the operation to complete
aks_target.wait_for_provisioning(True)```

### a. *Activate App Insights through updating AKS Webservice configuration*
In order to enable App Insights in your service you will need to update your AKS configuration file:

In [ ]:
# Set the web service configuration.
aks_deployment_config = AksWebservice.deploy_configuration(enable_app_insights=True)

### b. Deploy your service

In [ ]:
if aks_target.provisioning_state == "Succeeded":
    aks_service_name = "aks-service-appinsights"
    aks_service = Model.deploy(ws,
                               aks_service_name,
                               [model],
                               inference_config,
                               aks_deployment_config,
                               deployment_target=aks_target,
                               overwrite=True)
    aks_service.wait_for_deployment(show_output=True)
    print(aks_service.state)
else:
    raise ValueError("AKS cluster provisioning failed. Error: ", aks_target.provisioning_errors)

## 8. Test your service 

In [ ]:
%%time

if aks_service.state == "Healthy":
    test_sample = json.dumps({
        "data": [
            [1,28,13,45,54,6,57,8,8,10],
            [101,9,8,37,6,45,4,3,2,41]
        ]
    })

    prediction = aks_service.run(input_data=test_sample)
    print(prediction)
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aks_service.error)

## 9. See your service telemetry in App Insights
1. Go to the [Azure Portal](https://portal.azure.com/)
2. All resources--> Select the subscription/resource group where you created your Workspace--> Select the App Insights type
3. Click on the AppInsights resource. You'll see a highlevel dashboard with information on Requests, Server response time and availability.
4. Click on the top banner "Analytics"
5. In the "Schema" section select "traces" and run your query.
6. Voila! All your custom traces should be there.

# Disable App Insights

In [ ]:
aks_service.update(enable_app_insights=False)
aks_service.wait_for_deployment(show_output=True)

## Clean up

In [ ]:
%%time
aks_service.delete()
aci_service.delete()
model.delete()
if creating_compute:
    aks_target.delete()